# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from IPython.display import display

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [10]:
new_df=pd.read_csv("weather_data.csv")
new_df.head()

,Unnamed: 0,City,Country,Lat,Long,Date,Temp,Wind Spd,Cloud Cvr,Humid
0,0,Kapaa,US,22.0752,-159.3190,1637637507,300.14,1.34,90,76
1,1,Pisco,PE,-13.7000,-76.2167,1637637761,291.49,5.14,90,88
2,2,Tuktoyaktuk,CA,69.4541,-133.0374,1637637787,252.15,4.63,40,77
3,3,Grand Gaube,MU,-20.0064,57.6608,1637637763,297.80,0.45,1,66
4,4,Atuona,PF,-9.8000,-139.0333,1637637765,298.46,3.65,100,76


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = new_df[["Lat", "Long"]]
weight = new_df["Humid"]
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=weight, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [13]:
# Narrow down the DataFrame to find your ideal weather condition.
select_weather_df = new_df

# select_weather_df

# A max temperature lower than 80 degrees but higher than 70.
select_weather_df = select_weather_df.loc[(select_weather_df["Temp"] < 305) & (select_weather_df["Temp"] > 297)]

# Wind speed less than 10 mph.
select_weather_df = select_weather_df.loc[select_weather_df["Wind Spd"] < 5]

# # Zero cloudiness.
select_weather_df = select_weather_df.loc[select_weather_df["Cloud Cvr"] <100]

# Drop any rows with null values
select_weather_df = select_weather_df.dropna()

select_weather_df

,Unnamed: 0,City,Country,Lat,Long,Date,Temp,Wind Spd,Cloud Cvr,Humid
0,0,Kapaa,US,22.0752,-159.3190,1637637507,300.14,1.34,90,76
3,3,Grand Gaube,MU,-20.0064,57.6608,1637637763,297.80,0.45,1,66
28,28,Jérémie,HT,18.6500,-74.1167,1637637834,298.83,1.49,65,83
30,30,Labuhan,ID,-6.8844,112.2051,1637637811,302.93,1.75,94,66
33,33,Rikitea,PF,-23.1203,-134.9692,1637638153,297.53,4.81,1,74
...,...,...,...,...,...,...,...,...,...,...
555,555,Bubaque,GW,11.2833,-15.8333,1637638844,301.00,3.67,5,77
559,559,Beira,MZ,-19.8436,34.8389,1637638850,298.20,2.57,75,94
560,560,Saint-Paul,RE,-21.0096,55.2707,1637638851,299.20,0.42,35,69
564,564,Maceió,BR,-9.6658,-35.7353,1637638857,297.84,1.54,40,88


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [32]:
hotel_df = select_weather_df

params = {
    "radius": 5000,
    "types": "Hotels",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Long"]

    # changing location
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Hit API request
    print(f"Gathering Results for {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    try:
        print(f"Nearest place to stay is Hotel {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    #Exceptions    
    except (KeyError, IndexError):
        print("No Hotel in Area Found... skipping to next.")
        
    print("----------")

Gathering Results for 0: Kapaa.
A Hotel in the approximate area Kapaʻa.
----------
Gathering Results for 3: Grand Gaube.
A Hotel in the approximate area Goodlands.
----------
Gathering Results for 28: Jérémie.
A Hotel in the approximate area Jeremie.
----------
Gathering Results for 30: Labuhan.
A Hotel in the approximate area Musholla Babut Taubat.
----------
Gathering Results for 33: Rikitea.
A Hotel in the approximate area Rikitea.
----------
Gathering Results for 38: Yauco.
A Hotel in the approximate area Susúa Baja.
----------
Gathering Results for 40: Vaini.
A Hotel in the approximate area Nukuhetulu.
----------
Gathering Results for 47: Hobart.
A Hotel in the approximate area Hobart.
----------
Gathering Results for 52: Souillac.
A Hotel in the approximate area Souillac.
----------
Gathering Results for 53: Ouallam.
A Hotel in the approximate area Ouallam.
----------
Gathering Results for 55: Ténenkou.
A Hotel in the approximate area Tenenkou.
----------
Gathering Results for 56

A Hotel in the approximate area Saint-Paul.
----------
Gathering Results for 564: Maceió.
A Hotel in the approximate area Maceió.
----------
Gathering Results for 570: Mahébourg.
A Hotel in the approximate area Mahebourg.
----------


In [33]:
hotel_df.head(10)

,Unnamed: 0,City,Country,Lat,Long,Date,Temp,Wind Spd,Cloud Cvr,Humid,Hotel Name
0,0,Kapaa,US,22.0752,-159.3190,1637637507,300.14,1.34,90,76,Kapaʻa
3,3,Grand Gaube,MU,-20.0064,57.6608,1637637763,297.80,0.45,1,66,Goodlands
28,28,Jérémie,HT,18.6500,-74.1167,1637637834,298.83,1.49,65,83,Jeremie
30,30,Labuhan,ID,-6.8844,112.2051,1637637811,302.93,1.75,94,66,Musholla Babut Taubat
33,33,Rikitea,PF,-23.1203,-134.9692,1637638153,297.53,4.81,1,74,Rikitea
38,38,Yauco,PR,18.0350,-66.8499,1637637846,299.12,1.72,60,84,Susúa Baja
40,40,Vaini,TO,-21.2000,-175.2000,1637638161,302.24,2.57,40,79,Nukuhetulu
47,47,Hobart,AU,-42.8794,147.3294,1637637619,297.19,0.45,75,52,Hobart
52,52,Souillac,MU,-20.5167,57.5167,1637637863,297.79,3.09,40,88,Souillac
53,53,Ouallam,NE,14.3188,2.0866,1637637865,298.03,4.56,2,22,Ouallam


In [34]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Long"]]

In [35]:
# Add marker layer ontop of heat map

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

plt.savefig("images")

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>